In [61]:
import requests
import csv
from pathlib import Path
from dotenv import load_dotenv
import os
import pymongo

ModuleNotFoundError: No module named 'pymongo'

In [26]:
load_dotenv() 
WONDER_URL = os.environ['WONDER_URL']
TRIP_URL = os.environ['TRIP_URL']

comment_detail = {
        'symbol': symbol,
        'uuid': comment_uuid,
        'usr': username,
        'like': cmt_like,
        'time': rand_time,
        'text': str_comment.replace('  ', ' ')
    }
    tbl_comment.insert_one(comment_detail)


    tbl_stock.update_one({'symbol': symbol}, {'$set':{
                'name': stock_detail['name'],
                'industry': stock_detail['finnhubIndustry'],
                'ipo': stock_detail['ipo'],
                'cap': stock_detail['marketCapitalization'],
                'web': stock_detail['weburl']
            }})

            db_stock = tbl_stock.find_one({'name': None})


In [ ]:
db_client = pymongo.MongoClient('mongodb://localhost:27017')
collections = db_client['db_ai_travel_planner']
tb_city = collections['tb_city']

In [27]:
def custom_query(get_url):
    print(get_url)
    try:
        r = requests.get(get_url)
        return r.json()
    except Exception as e:
       print(e)
       return r

In [47]:
def post_request(keyword):
    url =  TRIP_URL + '20400/getGsMainSuggestForTripOnline'
    HEADER = {'Content-Type': 'application/json'}
    json_data = {
        "keyword": keyword.lower(),
        "head": {
            "extension": [
                {
                    "name": "locale",
                    "value": "en-US"
                },
                {
                    "name": "platform",
                    "value": "Online"
                },
                {
                    "name": "userAgent",
                    "value": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"
                }
            ]
        }
    }

    try:
        r = requests.post(url, headers=HEADER, json=json_data)
        return r.json()
    except Exception as e:
       print(e)
       return e

In [28]:
def load_csv(filepath):
    data = []
    with open(filepath, 'r', newline='', encoding='utf-8') as csvfile:  # Handle encoding!
        reader = csv.reader(csvfile)  # Or csv.DictReader for dictionaries
        header = next(reader)  # Read the header row (if it exists)
        for row in reader:
            data.append(row)  # Or data.append(dict(zip(header, row))) for DictReader
    return header, data  # Return header and data

In [ ]:
def search_city_in_trip(city):
    result = custom_query(TRIP_URL + 'api/v1/destinations?page=0&q=' + city.lower())  #1.8s

    return result['destinationMetas']

In [44]:
def search_city_in_wonderplan(city):
    result = custom_query(WONDER_URL + 'api/v1/destinations?page=0&q=' + city.lower())  #1.8s

    return result['destinationMetas']

In [54]:
#find common details between 2 services

def find_match_cities(city, country):
    response_data = {
        'city': city,
        'country': country,
        'wonder_id': '',    #ID in wonderland, sample: DE/BY/Munich
        'trip_id': 0,       #ID in trip, sample 1234
    }
    #1. find in wonderplan
    wonder_cities = search_city_in_wonderplan(city)
    if (len(wonder_cities) > 0):
        #found it in Wonderplan
        for item in wonder_cities:
            if item['type'] == 'DESTINATION_TYPE_CITY':
                response_data['wonder_id'] = item['id']
                #2. find in trip
                trip_cities = post_request(city)
                if trip_cities['data'] != None:
                    for trip_item in trip_cities['data']:
                        if trip_item['type'] == 'district':
                            if (city.lower() == trip_item['word'].lower().replace('<em>', '').replace('</em>', '')):
                                response_data['trip_id'] = trip_item['id']
                else:
                    print('Not found city in Trip: ' + city + ' country: ' + country)
    else:
        print('Not found city in Wonderplan: ' + city + ' country: ' + country)
    #2. find in trip

    #get common id

    return response_data

find_match_cities('Tokyo', 'Japan')

https://wonderplan.ai/api/v1/destinations?page=0&q=tokyo


{'city': 'Tokyo',
 'country': 'Japan',
 'wonder_id': 'JP/13/Tokyo',
 'trip_id': 294}

In [38]:
filepath = Path("./worldcities.csv")  # Relative path (better)

header, data = load_csv(filepath)

if header:
    print("Header:", header)
index = 0
for row in data:
    if index < 1:
        print(row[0])
        results = file_match_cities(row[0], row[4])
        print(results)
    index += 1

Header: ['city', 'city_ascii', 'lat', 'lng', 'country', 'iso2', 'iso3', 'admin_name', 'capital', 'population', 'id']
Tokyo
https://wonderplan.ai/api/v1/destinations?page=0&q=tokyo
{'city': 'Tokyo', 'country': 'Japan', 'wonder_id': '', 'trip_id': 0}
